- RoBerta
- https://vanche.github.io/spanbert_roberta/

- Electra ..._RTD
- https://blog.pingpong.us/electra-review/

- koelectra
- https://www.samsungsds.com/kr/insights/TechToolkit_2021_KoreALBERT.html

In [ ]:
!pip install adabelief-pytorch==0.2.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install transformers
import os
from imblearn.over_sampling import RandomOverSampler
import re
import numpy as np
from tqdm import tqdm
from glob import glob
import json
import requests
import tensorflow as tf
from transformers import BertModel, TFBertModel, TFRobertaModel, RobertaTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel, AutoModelForSequenceClassification
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold
from adabelief_pytorch import AdaBelief
from transformers.optimization import get_cosine_schedule_with_warmup
from tqdm import tqdm, tqdm_notebook
import shutil
import gc
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

     |████████████████████████████████| 3.4 MB 8.4 MB/s 
     |████████████████████████████████| 67 kB 6.3 MB/s 
     |████████████████████████████████| 895 kB 78.6 MB/s 
     |████████████████████████████████| 3.3 MB 89.0 MB/s 
     |████████████████████████████████| 596 kB 95.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
#random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)
BATCH_SIZE = 32
NUM_EPOCHS = 5 #

L_RATE = 1e-5
MAX_LEN = 32
max_grad_norm=1
log_interval=200
NUM_CORES = os.cpu_count()
device = torch.device("cuda:0")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator", cache_dir='bert_ckpt', do_lower_case=False)
#https://github.com/monologg/KoELECTRA

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/257k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

In [ ]:
# 데이터셋 불러오기
import pandas as pd
df = pd.read_csv('/content/drive/Shareddrives/BOAZ_Adv/sentiment_label4.csv') # df = pd.read_csv('/content/drive/Shareddrives/BOAZ_Adv/sentiment_label3.csv') 


In [ ]:
df

,content,label
0,와 강원진짜 대박이네...단장 능력오지네,1
1,대리점들도망하게하는..,3
2,결혼식에 내가 좋아하는 가수가 축가를 불러주기로 했어! 너무 기뻐.,0
3,이번 동아리 엠티는 선배들 없이 우리 학번과 후배들만 간다고 해. 너무 신나.,0
4,쉽고 간단하게 설명해 주셔서 감사홧팅!,0
...,...,...
68569,오늘은 새롭게 시작한 전공 시험 결과가 나왔어.,0
68570,남편이 나한테 어차피 죽을 거라며 치료를 포기하자고 해서 슬퍼.,2
68571,최여진의 저주인가??,1
68572,20번째 생일을 맞이하셔서 무지 기쁘실거예요.,0


In [ ]:
df.columns = ['title', 'topic_idx']
#train.topic_idx.value_counts()

In [ ]:
df.topic_idx.value_counts()

1    32171
2    18806
0    12767
3     4830
Name: topic_idx, dtype: int64

In [ ]:
#df['index'] = df.index

In [ ]:
df.head(5)

,title,topic_idx
0,와 강원진짜 대박이네...단장 능력오지네,1
1,대리점들도망하게하는..,3
2,결혼식에 내가 좋아하는 가수가 축가를 불러주기로 했어! 너무 기뻐.,0
3,이번 동아리 엠티는 선배들 없이 우리 학번과 후배들만 간다고 해. 너무 신나.,0
4,쉽고 간단하게 설명해 주셔서 감사홧팅!,0


In [ ]:
#df['index'] = df.index

In [ ]:
# train, test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(df , stratify=df['topic_idx']  ,test_size=0.25, random_state=0, shuffle = True)  #  stratify = y_label, // train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:
dataset_test = dataset_test.reset_index()
dataset_test = dataset_test[[ 'title', 'topic_idx']]
dataset_test

,title,topic_idx
0,동사자분들 너무 안타깝다.,2
1,어떤 여주와도 케미 짱!,3
2,나는 집에서 이렇게 누워 있고 싶은데 아들네가 꼭 요양 병원에 가자고 하네.,1
3,중국이 베트맨을 이길수있을까,3
4,스트레스를 받아도 참고만 살았는데 결국 병에 걸리고 말았어.,2
...,...,...
17139,돈이 든다는 사실 때문에 일을 안 하는 친구들은 만나기가 어려워.,2
17140,이거 해결 안되면 삼성은 큰 위기에 빠질 것은 불보듯 뻔하다,3
17141,김병만 연기잘하네,0
17142,아빠는 내가 하루 종일 공부만 하기를 원해서 너무 힘들어.,1


In [ ]:
### train 에 대해서면 randomoversampling 진행
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler() ###
titles_t = dataset_train.title.to_numpy().reshape(-1,1)
labels_t = dataset_train.topic_idx.to_numpy().reshape(-1,1)
labels_t=labels_t.astype('int')
X_over, y_over = oversample.fit_resample(titles_t, labels_t)
dataset_train = pd.DataFrame({'title' : X_over.reshape(-1),
                      'topic_idx' : y_over.reshape(-1)})
'''
### test 에 대해서면 randomoversampling 진행
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler() ###
titles_t = dataset_test.title.to_numpy().reshape(-1,1)
labels_t = dataset_test.topic_idx.to_numpy().reshape(-1,1)
labels_t=labels_t.astype('int')
X_over, y_over = oversample.fit_resample(titles_t, labels_t)
dataset_test = pd.DataFrame({'title' : X_over.reshape(-1),
                      'topic_idx' : y_over.reshape(-1)})
'''

"\n### test 에 대해서면 randomoversampling 진행\nfrom imblearn.over_sampling import RandomOverSampler\noversample = RandomOverSampler() ###\ntitles_t = dataset_test.title.to_numpy().reshape(-1,1)\nlabels_t = dataset_test.topic_idx.to_numpy().reshape(-1,1)\nlabels_t=labels_t.astype('int')\nX_over, y_over = oversample.fit_resample(titles_t, labels_t)\ndataset_test = pd.DataFrame({'title' : X_over.reshape(-1),\n                      'topic_idx' : y_over.reshape(-1)})\n"

In [ ]:
dataset_test.topic_idx.value_counts()

1    8043
2    4702
0    3192
3    1207
Name: topic_idx, dtype: int64

In [ ]:
dataset_train.topic_idx.value_counts()

3    24128
2    24128
1    24128
0    24128
Name: topic_idx, dtype: int64

In [ ]:

# Koelectra 모델의 입력으로 들어갈 수 있게 형태 변환해주는 class
class TrainDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
        self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator", cache_dir='bert_ckpt', do_lower_case=False) #
    def __getitem__(self, index):
        # get the sentence from the dataframe
        sentence = self.df_data.loc[index, 'title'] # sentence = self.df_data[self.title].iloc[index] # 
        encoded_dict = tokenizer(
          text = sentence,
          add_special_tokens = True, 
          max_length = MAX_LEN,
          pad_to_max_length = True,
          truncation=True,           # Pad & truncate all sentences.
          return_tensors="pt")

        padded_token_list = encoded_dict['input_ids'][0]
        token_type_id = encoded_dict['token_type_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        target = torch.tensor(self.df_data.loc[index, "topic_idx"]) # target = torch.tensor(self.df_data[self.topic_idx].iloc[index]) # # target = torch.tensor(self.df_data[self.topic_idx].iloc[index]) #   # 
        sample = (padded_token_list, token_type_id , att_mask, target)
        return sample
    def __len__(self):
        return len(self.df_data)



'''
class TestDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
    def __getitem__(self, index):
        # get the sentence from the dataframe
        sentence = self.df_data.loc[index, 'title'] # sentence = self.df_data[self.title].iloc[index] #
        encoded_dict = tokenizer(
          text = sentence,
          add_special_tokens = True, 
          max_length = MAX_LEN,
          pad_to_max_length = True,
          truncation=True,           # Pad & truncate all sentences.
          return_tensors="pt")

        padded_token_list = encoded_dict['input_ids'][0]
        token_type_id = encoded_dict['token_type_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        sample = (padded_token_list, token_type_id , att_mask)
        return sample
    def __len__(self):
        return len(self.df_data)
'''

'\nclass TestDataset(Dataset):\n    def __init__(self, df):\n        self.df_data = df\n    def __getitem__(self, index):\n        # get the sentence from the dataframe\n        sentence = self.df_data.loc[index, \'title\'] # sentence = self.df_data[self.title].iloc[index] #\n        encoded_dict = tokenizer(\n          text = sentence,\n          add_special_tokens = True, \n          max_length = MAX_LEN,\n          pad_to_max_length = True,\n          truncation=True,           # Pad & truncate all sentences.\n          return_tensors="pt")\n\n        padded_token_list = encoded_dict[\'input_ids\'][0]\n        token_type_id = encoded_dict[\'token_type_ids\'][0]\n        att_mask = encoded_dict[\'attention_mask\'][0]\n        sample = (padded_token_list, token_type_id , att_mask)\n        return sample\n    def __len__(self):\n        return len(self.df_data)\n'

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
###
train_data =TrainDataset(dataset_train) 
test_data = TrainDataset(dataset_test)

In [ ]:
# train_data = TrainDataset(train)

train_dataloader = torch.utils.data.DataLoader(train_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                      num_workers=NUM_CORES)
test_dataloader = torch.utils.data.DataLoader(test_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=False,
                                      num_workers=NUM_CORES)


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator",num_labels=4)


####미세조정
n=0
for name, child in model.named_children():
    if n==0:
      h=0
      for param in child.parameters():
        if h<=328: #이부분 숫자 조절로 fine-tuning => Roberta229: h=229
          param.requires_grad = False
        h+=1
    n+=1
#####
    # print(param)
model.to(device)
optimizer = AdaBelief(model.parameters(), lr=1e-5, eps=1e-16, betas=(0.9,0.999), weight_decouple = True, rectify = False)

warmup_ratio = 0.1
t_total = len(train_dataloader) * NUM_EPOCHS
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


Downloading:   0%|          | 0.00/431M [00:00<?, ?B/s]

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief


In [ ]:
from tqdm import tqdm
for e in range(NUM_EPOCHS):
    train_acc = 0.0
    test_acc = 0.0
    best_acc =0.0
    model.train()
    torch.set_grad_enabled(True)
    for batch_id, (input_id,token_type_id,attention_mask,label) in enumerate(tqdm(train_dataloader)) : # enumerate(tqdm_notebook(train_dataloader))
        optimizer.zero_grad()
        input_id = input_id.long().to(device)
        token_type_id = token_type_id.long().to(device)
        attention_mask = attention_mask.long().to(device)
        label = label.to(device)
        outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask, labels=label)
        loss = outputs[0]
        out = outputs[1]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} train loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (input_id,token_type_id,attention_mask,label) in enumerate(tqdm(test_dataloader)) :
            input_id = input_id.long().to(device)
            token_type_id = token_type_id.long().to(device)
            attention_mask = attention_mask.long().to(device)
            label = label.to(device)
            outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask, labels=label)
            loss = outputs[0]
            out = outputs[1]
            #loss.backward()
            #torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            #optimizer.step()
            #scheduler.step()  # Update learning rate schedule
            
            test_acc += calc_accuracy(out, label)
            if batch_id % log_interval == 0:
                print("epoch {} batch id {} test loss {} test acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), test_acc / (batch_id+1)))
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

        #print("Train_loss :", Train_loss, "Train_acc :",Train_acc , "Test_loss:", Test_loss  , "Test_acc:", Test_acc)
    

  0%|          | 0/3016 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_lengt

epoch 1 batch id 1 train loss 1.4072840213775635 train acc 0.15625


  7%|▋         | 205/3016 [00:08<01:46, 26.40it/s]

epoch 1 batch id 201 train loss 1.3916664123535156 train acc 0.2660136815920398


 13%|█▎        | 406/3016 [00:15<01:38, 26.46it/s]

...

epoch 5 batch id 201 test loss 1.243461012840271 test acc 0.37842039800995025


 76%|███████▌  | 406/536 [00:15<00:04, 27.28it/s]

epoch 5 batch id 401 test loss 1.1353994607925415 test acc 0.3724283042394015


100%|██████████| 536/536 [00:19<00:00, 26.99it/s]

epoch 5 test acc 0.3762049129353234


In [ ]:

## 학습 모델 저장
PATH = '/content/drive/Shareddrives/BOAZ_Adv/nlp_model' 
torch.save(model, PATH + 'Koelecta-base_model_01월14일_epoch5.pt')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'Koelecta-base_model_01월14일_epoch5_state_dict.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'Koelecta-base_model_01월14일_epoch5_optimizer_all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

In [ ]:

# key error이 왜날가^^
# https://github.com/google-research/electra/issues/76
# 
# https://velog.io/@na2na8/ELECTRA%EB%A1%9C-Binary-Classification